# <center>Introduction to Curricular Simulations</center>

<center>
    <b>Gregory L. Heileman$^\dagger$, Jiacheng Zhang$^\ddagger$ and Hayden W. Free$^\bullet$</b> <br>
    $^\dagger$Department of Electrical & Computer Engineering <br>
    University of Arizona <br>
    heileman@arizona.edu <br>
    $^\ddagger$Department of Computer Science <br>
    jiachengzhang1@arizona.edu <br>
    University of Arizona<br>
    $^\bullet$Department of Computer Science <br>
    hayden.free@uky.edu <br>
    University of Kentucky
</center>

## 1. Introduction

This notebook demonstrates how to use the simulation capabilites that are included as a part of the [CurricularAnalytics toolbox](https://github.com/CurricularAnalytics/CurricularAnalytics.jl). If you would like to become more familiar with the notions behind curricular analytics, we suggest you read <cite data-cite="he:18">Heileman, et. al, (2018)</cite>, and also examine the Introduction to the Curricular Analytics Toolbox notebook that accompanies this notebook.

The simulation capabilites include the ability to simulate the flow of students through a curriculum, towards graduation, using discrete event simulation. Specifically, a population of students attempt to complete the selected curriculum, by taking courses in the order prescribed by the curriculum. At each step (semester) of the simulation a given student enrolls in a set of courses, earning either a passing of failing grade in each.  At the end of a given semester, if a student has passed all of the courses in the curriculum, they are deemed a graduate.  If a student has not yet gradauted, then they may stop out (according to a prescribed stop-out model), or enroll in the next set of courses available to them. One of the intended uses of these simulation capabilities is to estimate the impact that particular curricular changes or instructional improvements will have on student progress. 

The simulation framework (shown below) was orginally developed by <cite data-cite="hi:18">Hickman, (2014)</cite>, and  subsequent development has allowed it to be integrated into the CurricularAnalytics toolbox. 

<img src="SimulationFramework.png" width="600">

Notice that the simulation "engine" requires three inputs: a curriculum, a model for students, and a model for student peformance. The results of the simualtion are returned in an object that may be viewed using the `simulation_report()` function, as demonstrated below. 

In order to perform curricular simulations, first load the Curricular Analytics toolbox modules:

In [1]:
using CurricularAnalytics, CurricularVisualization

HTML{String}("<script>\n// Immediately-invoked-function-expression to avoid global variables.\n(function() {\n    var warning_div = document.getElementById(\"webio-warning-4960100937364793638\");\n    var hide = function () {\n        var script = document.getElementById(\"webio-setup-7787284047229102567\");\n        var parent = script && script.parentElement;\n        var grandparent = parent && parent.parentElement;\n        if (grandparent) {\n            grandparent.style.display = \"none\";\n        }\n        warning_div.style.display = \"none\";\n    };\n    if (typeof Jupyter !== \"undefined\") {\n        console.log(\"WebIO detected Jupyter notebook environment.\");\n        // Jupyter notebook.\n        var extensions = (\n            Jupyter\n            && Jupyter.notebook.config.data\n            && Jupyter.notebook.config.data.load_extensions\n        );\n        if (extensions && extensions[\"webio-jupyter-notebook\"]) {\n            // Extension already loaded.\n            console.log(\"Jupyter WebIO nbextension detected; not loading ad-hoc.\");\n            hide();\n            return;\n        }\n    } else if (window.location.pathname.includes(\"/lab\")) {\n        // Guessing JupyterLa\n        console.log(\"Jupyter Lab detected; make sure the @webio/jupyter-lab-provider labextension is installed.\");\n        hide();\n        return;\n    }\n})();\n\n</script>\n<p\n    id=\"webio-warning-4960100937364793638\"\n    class=\"output_text output_stderr\"\n    style=\"padding: 1em; font-weight: bold;\"\n>\n    Unable to load WebIO. Please make sure WebIO works for your Jupyter client.\n    For troubleshooting, please see <a href=\"https://juliagizmos.github.io/WebIO.jl/latest/providers/ijulia/\">\n    the WebIO/IJulia documentation</a>.\n    <!-- TODO: link to installation docs. -->\n</p>\n")

HTML{String}("<script>\n// Immediately-invoked-function-expression to avoid global variables.\n(function() {\n    var warning_div = document.getElementById(\"webio-warning-11570808977089851273\");\n    var hide = function () {\n        var script = document.getElementById(\"webio-setup-12895921644868037696\");\n        var parent = script && script.parentElement;\n        var grandparent = parent && parent.parentElement;\n        if (grandparent) {\n            grandparent.style.display = \"none\";\n        }\n        warning_div.style.display = \"none\";\n    };\n    if (typeof Jupyter !== \"undefined\") {\n        console.log(\"WebIO detected Jupyter notebook environment.\");\n        // Jupyter notebook.\n        var extensions = (\n            Jupyter\n            && Jupyter.notebook.config.data\n            && Jupyter.notebook.config.data.load_extensions\n        );\n        if (extensions && extensions[\"webio-jupyter-notebook\"]) {\n            // Extension already loaded.\n            console.log(\"Jupyter WebIO nbextension detected; not loading ad-hoc.\");\n            hide();\n            return;\n        }\n    } else if (window.location.pathname.includes(\"/lab\")) {\n        // Guessing JupyterLa\n        console.log(\"Jupyter Lab detected; make sure the @webio/jupyter-lab-provider labextension is installed.\");\n        hide();\n        return;\n    }\n})();\n\n</script>\n<p\n    id=\"webio-warning-11570808977089851273\"\n    class=\"output_text output_stderr\"\n    style=\"padding: 1em; font-weight: bold;\"\n>\n    Unable to load WebIO. Please make sure WebIO works for your Jupyter client.\n    For troubleshooting, please see <a href=\"https://juliagizmos.github.io/WebIO.jl/latest/providers/ijulia/\">\n    the WebIO/IJulia documentation</a>.\n    <!-- TODO: link to installation docs. -->\n</p>\n")

## 2. Setting up the Simulation Environment

The first thing we will do is read in a degree plan. The student in the simulation will attempt to complete the curriculum associated with the degree plan in the order that prescribed in the degree plan.  Specifically, in each semester each student will enroll in courses they have not yet taken in the order specified by the degree plan until they reach the maximum allowed number of credit hours.

### 2.1 Reading the Degree Plan

The following commands read in a degree plan stored in the CSV file format, and then display a visualization of the resulting plan.

In [2]:
AE_degree_plan = read_csv("Univ_of_Arizona-Aero.csv")
visualize(AE_degree_plan, notebook=true, scale=0.8)

WebIO.Scope(WebIO.Node{WebIO.DOM}(WebIO.DOM(:html, :iframe), Any[""], Dict{Symbol,Any}(:src => "http://localhost:8156",:id => "curriculum",:events => Dict{Symbol,WebIO.JSString}(:load => WebIO.JSString("(function (){this.contentWindow.postMessage({\"options\":{\"hideTerms\":false,\"edit\":false},\"curriculum\":{\"name\":\"Aerospace Engineering\",\"dp_name\":\"2019-20 Degree Plan\",\"curriculum_terms\":[{\"name\":\"Term 1\",\"curriculum_items\":[{\"curriculum_requisites\":[],\"name\":\"MATH 125\",\"metrics\":{\"centrality\":0,\"complexity\":38.0,\"blocking factor\":29,\"delay factor\":9.0},\"nameSub\":\"Calculus I w/ Applications\",\"id\":1,\"credits\":3,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\"CHEM 151\",\"metrics\":{\"centrality\":0,\"complexity\":5.0,\"blocking factor\":2,\"delay factor\":3.0},\"nameSub\":\"Gen Chemistry I\",\"id\":2,\"credits\":4,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\"ENGL 101\",\"metrics\":{\"centrality\":0,\"complexity\":3.0,\"blocking factor\":1,\"delay factor\":2.0},\"nameSub\":\"First-year Comp.\",\"id\":3,\"credits\":3,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[{\"source_id\":1,\"target_id\":4,\"type\":\"CurriculumCorequisite\"}],\"name\":\"ENGR 102\",\"metrics\":{\"centrality\":0,\"complexity\":2.0,\"blocking factor\":0,\"delay factor\":2.0},\"nameSub\":\"Intro to Engineering\",\"id\":4,\"credits\":3,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\" \",\"metrics\":{\"centrality\":0,\"complexity\":1.0,\"blocking factor\":0,\"delay factor\":1.0},\"nameSub\":\"Tier I General Ed.\",\"id\":5,\"credits\":3,\"nameCanonical\":\"\"}],\"id\":1},{\"name\":\"Term 2\",\"curriculum_items\":[{\"curriculum_requisites\":[{\"source_id\":1,\"target_id\":6,\"type\":\"CurriculumPrerequisite\"}],\"name\":\"MATH 129\",\"metrics\":{\"centrality\":387,\"complexity\":31.0,\"blocking factor\":22,\"delay factor\":9.0},\"nameSub\":\"Calculus II\",\"id\":6,\"credits\":3,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[{\"source_id\":1,\"target_id\":7,\"type\":\"CurriculumPrerequisite\"}],\"name\":\"AME 105\",\"metrics\":{\"centrality\":28,\"complexity\":16.0,\"blocking factor\":9,\"delay factor\":7.0},\"nameSub\":\"Intro. to MATLAB I\",\"id\":7,\"credits\":1,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[{\"source_id\":1,\"target_id\":8,\"type\":\"CurriculumPrerequisite\"}],\"name\":\"PHYS 141\",\"metrics\":{\"centrality\":277,\"complexity\":28.0,\"blocking factor\":19,\"delay factor\":9.0},\"nameSub\":\"Intro Mechanics\",\"id\":8,\"credits\":4,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[{\"source_id\":3,\"target_id\":9,\"type\":\"CurriculumPrerequisite\"}],\"name\":\"ENGL 102\",\"metrics\":{\"centrality\":0,\"complexity\":2.0,\"blocking factor\":0,\"delay factor\":2.0},\"nameSub\":\"First-year Comp.\",\"id\":9,\"credits\":3,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[{\"source_id\":1,\"target_id\":10,\"type\":\"CurriculumCorequisite\"}],\"name\":\"ECE 175\",\"metrics\":{\"centrality\":0,\"complexity\":2.0,\"blocking factor\":0,\"delay factor\":2.0},\"nameSub\":\"Programming for Eng. Apps.\",\"id\":10,\"credits\":3,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\" \",\"metrics\":{\"centrality\":0,\"complexity\":1.0,\"blocking factor\":0,\"delay factor\":1.0},\"nameSub\":\"Tier I General Ed. 1\",\"id\":11,\"credits\":3,\"nameCanonical\":\"\"}],\"id\":2},{\"name\":\"Term 3\",\"curriculum_items\":[{\"curriculum_requisites\":[{\"source_id\":8,\"target_id\":12,\"type\":\"CurriculumPrerequisite\"},{\"source_id\":6,\"target_id\":12,\"type\":\"CurriculumPrerequisite\"}],\"name\":\"CE 214\",\"metrics\":{\"centrality\":264,\"complexity\":24.0,\"blocking factor\":15,\"delay factor\":9.0},\"nameSub\":\"Statics\",\"id\":12,\"credits\":3,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[{\"source_id\":6,\"target_id\":13,\"type\":\"CurriculumPrerequisite\"}],\"name\":\"MATH  223\",\"metrics\":{\"centrality\":252,\"complexity\":25

In [3]:
basic_metrics(AE_degree_plan)
AE_degree_plan.metrics

Dict{String,Any} with 8 entries:
  "total credit hours"         => 129
  "avg. credits per term"      => 16.125
  "min. credits in a term"     => 15
  "term credit hour std. dev." => 0.927025
  "number of terms"            => 8
  "max. credits in a term"     => 18
  "min. credit term"           => 4
  "max. credit term"           => 3

In [4]:
CS_degree_plan = read_csv("Univ_of_Arizona-CS.csv")
visualize(CS_degree_plan, notebook=true, scale=0.8)

WebIO.Scope(WebIO.Node{WebIO.DOM}(WebIO.DOM(:html, :iframe), Any[""], Dict{Symbol,Any}(:src => "http://localhost:8156",:id => "curriculum",:events => Dict{Symbol,WebIO.JSString}(:load => WebIO.JSString("(function (){this.contentWindow.postMessage({\"options\":{\"hideTerms\":false,\"edit\":false},\"curriculum\":{\"name\":\"Computer Science\",\"dp_name\":\"2019-20 Degree Plan\",\"curriculum_terms\":[{\"name\":\"Term 1\",\"curriculum_items\":[{\"curriculum_requisites\":[{\"source_id\":2,\"target_id\":1,\"type\":\"CurriculumCorequisite\"}],\"name\":\"CSC 110\",\"metrics\":{\"centrality\":25,\"complexity\":12.0,\"blocking factor\":7,\"delay factor\":5.0},\"nameSub\":\"Intro to Computer Programming I\",\"id\":1,\"credits\":4,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\"MATH 120R\",\"metrics\":{\"centrality\":0,\"complexity\":15.0,\"blocking factor\":10,\"delay factor\":5.0},\"nameSub\":\"Pre-Calculus\",\"id\":2,\"credits\":4,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\"ENGL 101\",\"metrics\":{\"centrality\":0,\"complexity\":3.0,\"blocking factor\":1,\"delay factor\":2.0},\"nameSub\":\"English Composition I\",\"id\":3,\"credits\":3,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\" \",\"metrics\":{\"centrality\":0,\"complexity\":3.0,\"blocking factor\":1,\"delay factor\":2.0},\"nameSub\":\"1st Semester Second Language\",\"id\":4,\"credits\":4,\"nameCanonical\":\"\"}],\"id\":1},{\"name\":\"Term 2\",\"curriculum_items\":[{\"curriculum_requisites\":[{\"source_id\":1,\"target_id\":5,\"type\":\"CurriculumPrerequisite\"}],\"name\":\"CSC 120\",\"metrics\":{\"centrality\":25,\"complexity\":11.0,\"blocking factor\":6,\"delay factor\":5.0},\"nameSub\":\"Intro to Computer Programming II\",\"id\":5,\"credits\":4,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[{\"source_id\":2,\"target_id\":6,\"type\":\"CurriculumPrerequisite\"}],\"name\":\"MATH 122A/B\",\"metrics\":{\"centrality\":3,\"complexity\":4.0,\"blocking factor\":1,\"delay factor\":3.0},\"nameSub\":\"Calculus I\",\"id\":6,\"credits\":5,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[{\"source_id\":3,\"target_id\":7,\"type\":\"CurriculumPrerequisite\"}],\"name\":\"ENGL 102\",\"metrics\":{\"centrality\":0,\"complexity\":2.0,\"blocking factor\":0,\"delay factor\":2.0},\"nameSub\":\"English Composition II\",\"id\":7,\"credits\":4,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[{\"source_id\":4,\"target_id\":8,\"type\":\"CurriculumPrerequisite\"}],\"name\":\" \",\"metrics\":{\"centrality\":0,\"complexity\":2.0,\"blocking factor\":0,\"delay factor\":2.0},\"nameSub\":\"2nd Semester Second Language\",\"id\":8,\"credits\":4,\"nameCanonical\":\"\"}],\"id\":2},{\"name\":\"Term 3\",\"curriculum_items\":[{\"curriculum_requisites\":[{\"source_id\":5,\"target_id\":9,\"type\":\"CurriculumPrerequisite\"}],\"name\":\"CSC 210\",\"metrics\":{\"centrality\":20,\"complexity\":9.0,\"blocking factor\":4,\"delay factor\":5.0},\"nameSub\":\"Software Development\",\"id\":9,\"credits\":4,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[{\"source_id\":5,\"target_id\":10,\"type\":\"CurriculumPrerequisite\"}],\"name\":\"CSC 245\",\"metrics\":{\"centrality\":5,\"complexity\":6.0,\"blocking factor\":1,\"delay factor\":5.0},\"nameSub\":\"Intro to Discrete Structures\",\"id\":10,\"credits\":3,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[{\"source_id\":6,\"target_id\":11,\"type\":\"CurriculumPrerequisite\"}],\"name\":\"MATH 129\",\"metrics\":{\"centrality\":0,\"complexity\":3.0,\"blocking factor\":0,\"delay factor\":3.0},\"nameSub\":\"Calculus II\",\"id\":11,\"credits\":3,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\" \",\"metrics\":{\"centrality\":0,\"complexity\":1.0,\"blocking factor\":0,\"delay factor\":1.0},\"nameSub\":\"Tier 1 General Education 1\",\"id\":12,\"credits\":3,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\" \",\"metrics\":{\"centrality\":0,\"complexity\":1.0,\"blocking factor\":0,\"delay facto

In [5]:
EE_degree_plan = read_csv("Ga_Tech-EE.csv")
visualize(EE_degree_plan, notebook=true, scale=0.8)

WebIO.Scope(WebIO.Node{WebIO.DOM}(WebIO.DOM(:html, :iframe), Any[""], Dict{Symbol,Any}(:src => "http://localhost:8156",:id => "curriculum",:events => Dict{Symbol,WebIO.JSString}(:load => WebIO.JSString("(function (){this.contentWindow.postMessage({\"options\":{\"hideTerms\":false,\"edit\":false},\"curriculum\":{\"name\":\"Electrical Engineering\",\"dp_name\":\"2017-18 Plan\",\"curriculum_terms\":[{\"name\":\"Term 1\",\"curriculum_items\":[{\"curriculum_requisites\":[],\"name\":\" \",\"metrics\":{\"centrality\":0,\"complexity\":9.0,\"blocking factor\":4,\"delay factor\":5.0},\"nameSub\":\"CHEM 1310\",\"id\":1,\"credits\":4,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\" \",\"metrics\":{\"centrality\":0,\"complexity\":23.0,\"blocking factor\":15,\"delay factor\":8.0},\"nameSub\":\"MATH 1551\",\"id\":2,\"credits\":2,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\" \",\"metrics\":{\"centrality\":0,\"complexity\":16.0,\"blocking factor\":10,\"delay factor\":6.0},\"nameSub\":\"MATH 1553\",\"id\":3,\"credits\":2,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\" \",\"metrics\":{\"centrality\":0,\"complexity\":17.0,\"blocking factor\":10,\"delay factor\":7.0},\"nameSub\":\"CS 1371\",\"id\":4,\"credits\":3,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\" \",\"metrics\":{\"centrality\":0,\"complexity\":1.0,\"blocking factor\":0,\"delay factor\":1.0},\"nameSub\":\"Wellness Elective\",\"id\":5,\"credits\":3,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\" \",\"metrics\":{\"centrality\":0,\"complexity\":3.0,\"blocking factor\":1,\"delay factor\":2.0},\"nameSub\":\"ENGL 1101\",\"id\":6,\"credits\":3,\"nameCanonical\":\"\"}],\"id\":1},{\"name\":\"Term 2\",\"curriculum_items\":[{\"curriculum_requisites\":[{\"source_id\":2,\"target_id\":7,\"type\":\"CurriculumPrerequisite\"}],\"name\":\" \",\"metrics\":{\"centrality\":29,\"complexity\":17.0,\"blocking factor\":9,\"delay factor\":8.0},\"nameSub\":\"PHYS 2211\",\"id\":7,\"credits\":4,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[{\"source_id\":2,\"target_id\":8,\"type\":\"CurriculumPrerequisite\"}],\"name\":\" \",\"metrics\":{\"centrality\":34,\"complexity\":17.0,\"blocking factor\":10,\"delay factor\":7.0},\"nameSub\":\"MATH 1552\",\"id\":8,\"credits\":4,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[{\"source_id\":4,\"target_id\":9,\"type\":\"CurriculumPrerequisite\"}],\"name\":\" \",\"metrics\":{\"centrality\":11,\"complexity\":13.0,\"blocking factor\":6,\"delay factor\":7.0},\"nameSub\":\"ECE 2020\",\"id\":9,\"credits\":3,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\" \",\"metrics\":{\"centrality\":0,\"complexity\":1.0,\"blocking factor\":0,\"delay factor\":1.0},\"nameSub\":\"History/Government Elective\",\"id\":10,\"credits\":3,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[{\"source_id\":6,\"target_id\":11,\"type\":\"CurriculumPrerequisite\"}],\"name\":\" \",\"metrics\":{\"centrality\":0,\"complexity\":2.0,\"blocking factor\":0,\"delay factor\":2.0},\"nameSub\":\"ENGL 1102\",\"id\":11,\"credits\":3,\"nameCanonical\":\"\"}],\"id\":2},{\"name\":\"Term 3\",\"curriculum_items\":[{\"curriculum_requisites\":[{\"source_id\":7,\"target_id\":12,\"type\":\"CurriculumPrerequisite\"}],\"name\":\" \",\"metrics\":{\"centrality\":29,\"complexity\":16.0,\"blocking factor\":8,\"delay factor\":8.0},\"nameSub\":\"PHYS 2212\",\"id\":12,\"credits\":4,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[{\"source_id\":3,\"target_id\":13,\"type\":\"CurriculumPrerequisite\"},{\"source_id\":8,\"target_id\":13,\"type\":\"CurriculumPrerequisite\"}],\"name\":\" \",\"metrics\":{\"centrality\":0,\"complexity\":3.0,\"blocking factor\":0,\"delay factor\":3.0},\"nameSub\":\"MATH 2552\",\"id\":13,\"credits\":4,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[{\"source_id\":4,\"target_id\":14,\"type\":\"CurriculumPrerequisite\"},{\"source_id\":3,\"target_id\":14,\"type\":\"CurriculumPrerequisite\"},{\"sou

In [6]:
basic_metrics(EE_degree_plan)
EE_degree_plan.metrics

Dict{String,Any} with 8 entries:
  "total credit hours"         => 134
  "avg. credits per term"      => 16.75
  "min. credits in a term"     => 15
  "term credit hour std. dev." => 1.08972
  "number of terms"            => 8
  "max. credits in a term"     => 18
  "min. credit term"           => 4
  "max. credit term"           => 5

### 2.2 Creating the Student Cohort

The following command will create an inital cohort of students `n` using a simple enrollment model. Specifically, with this simple model, all students are assumed equally likely (or unlikely) to pass a given class according the course pass rate probability model.

In [7]:
enrollment_model = Enrollment  # use the Enrollment module to determine if/when student may enroll in a course
stopouts = true  # assume that student may stop out of the cohort
n = 1000   # student cohort size will be 100
students = simple_students(n);  # create a student cohort

### 2.3 The Course Performance Model

First let's see what happens if the instructional complexity is the same for both programs. Specifically, we will use course pass rates as a proxy for instructional complexity, and we'll set the pass rates for all courses at 90%. 

In [59]:
performance_model = PassRate
real_passrate = false
set_passrates(AE_degree_plan.curriculum.courses, 0.9)
set_passrates(CS_degree_plan.curriculum.courses, 0.9)
course_passrate = 0.9;

### 2.4 Setting the Simulation Parameters

In [60]:
max_credits = 18  # the maximum number of credit hours a student may enroll in during a semester
duration_lock = false # rather than simulating until no students are left in the cohort, run for a fixed number of terms
num_terms = 12  # the maximum number of terms in the simulation
course_attempt_limit = 3;  # number of times a student may attempt a course

### 2.5 Running the Simulation 
The `simulation` function is used to execute the simulation.  Depending upon how many students are in the cohort, this may take some time to run.

In [61]:
simulation = simulate(AE_degree_plan, course_attempt_limit, students,
                      max_credits = max_credits,
                      performance_model = performance_model,
                      enrollment_model = enrollment_model,
                      duration = num_terms,
                      duration_lock = duration_lock,
                      stopouts = stopouts);

In order to view the results of the simulation, use the `simulation_report` function:

In [62]:
simulation_report(simulation, num_terms, course_passrate, max_credits, real_passrate)


------------ Simulation Report ------------
Aerospace Engineering, BS -- 2019-20 Degree Plan

-------- Simulation Statistics --------
Number of terms: 12
Max Credits per Term: 18
Max Course Attempts: 3
Number of Students: 1000
Preset Course Pass Rates: 90.0%

-------- Graduation Statistics --------
Number of Students Graduated: 657
Graduation Rate: 65.7%
Term Graduation Rates: 
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.045, 0.358, 0.592, 0.651, 0.657]
Average time to degree: 9.494672754946727 terms

-------- Stop out Statistics --------
Number of Students Stopped Out (Stopout Model Prediction + Reached Max Attempts): 343
Number of Students Reaching Max Attempts: 34
Stop-out Rate: 34.3%
Cumulative Term Stop-out Rates (including reached max course attempts students): 
[0.085, 0.199, 0.233, 0.28, 0.31, 0.326, 0.331, 0.334, 0.337, 0.341, 0.343, 0.343]

Cumulative Term Stop-out Rates (excluding reaching max course attempts students): 
[0.085, 0.199, 0.228, 0.272, 0.294, 0.304, 0.309, 0.309, 0.

Now let's run the same set of students through the Computer Science curriculum:

In [74]:
simulation = simulate(CS_degree_plan, course_attempt_limit, students,
                      max_credits = max_credits,
                      performance_model = performance_model,
                      enrollment_model = enrollment_model,
                      duration = num_terms,
                      duration_lock = duration_lock,
                      stopouts = stopouts);
simulation_report(simulation, num_terms, course_passrate, max_credits, real_passrate)


------------ Simulation Report ------------
Computer Science, BS -- 2019-20 Degree Plan

-------- Simulation Statistics --------
Number of terms: 12
Max Credits per Term: 18
Max Course Attempts: 3
Number of Students: 1000
Preset Course Pass Rates: 90.0%

-------- Graduation Statistics --------
Number of Students Graduated: 624
Graduation Rate: 62.4%
Term Graduation Rates: 
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.197, 0.532, 0.619, 0.624, 0.0]
Average time to degree: 8.83974358974359 terms

-------- Stop out Statistics --------
Number of Students Stopped Out (Stopout Model Prediction + Reached Max Attempts): 376
Number of Students Reaching Max Attempts: 77
Stop-out Rate: 37.6%
Cumulative Term Stop-out Rates (including reached max course attempts students): 
[0.077, 0.184, 0.231, 0.297, 0.336, 0.355, 0.368, 0.372, 0.373, 0.375, 0.376, 0.0]

Cumulative Term Stop-out Rates (excluding reaching max course attempts students): 
[0.077, 0.184, 0.212, 0.271, 0.29, 0.293, 0.299, 0.299, 0.299, 0.29

In [72]:
real_passrate = false
set_passrates(EE_degree_plan.curriculum.courses, 0.9)
simulation = simulate(EE_degree_plan, course_attempt_limit, students,
                      max_credits = max_credits,
                      performance_model = performance_model,
                      enrollment_model = enrollment_model,
                      duration = num_terms,
                      duration_lock = duration_lock,
                      stopouts = stopouts);
simulation_report(simulation, num_terms, course_passrate, max_credits, real_passrate)


------------ Simulation Report ------------
Electrical Engineering, BS -- 2017-18 Plan

-------- Simulation Statistics --------
Number of terms: 12
Max Credits per Term: 18
Max Course Attempts: 3
Number of Students: 1000
Preset Course Pass Rates: 90.0%

-------- Graduation Statistics --------
Number of Students Graduated: 680
Graduation Rate: 68.0%
Term Graduation Rates: 
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.061, 0.425, 0.626, 0.676, 0.68]
Average time to degree: 9.370588235294118 terms

-------- Stop out Statistics --------
Number of Students Stopped Out (Stopout Model Prediction + Reached Max Attempts): 320
Number of Students Reaching Max Attempts: 37
Stop-out Rate: 32.0%
Cumulative Term Stop-out Rates (including reached max course attempts students): 
[0.07, 0.19, 0.218, 0.262, 0.284, 0.297, 0.304, 0.307, 0.313, 0.319, 0.32, 0.32]

Cumulative Term Stop-out Rates (excluding reaching max course attempts students): 
[0.07, 0.19, 0.211, 0.248, 0.27, 0.278, 0.283, 0.283, 0.283, 0.283, 

Next we will use a model to determine whether or not a student passes a course. We will use actual pass/fail rates computed using historical data for the courses in the degree plan shown above. 

In [65]:
course_passrate = 0.9  # use if a course is not contained in the CSV file
real_passrate = true  # use the actual pass rates, rather than course_passrate for all courses
set_passrates_from_csv(AE_degree_plan.curriculum.courses, "./Student_Grades_sp17_to_fall19.csv", course_passrate)

Note: A more realistic model for predicting student performance could be used here. Specifically, a more realistic model might:

- take student demographics into account, including the major they are in,
- take prior grades into account when predicting future grades,
- take into account factors that influence student stopout, e.g., academic standing, GPA, unment need, etc.

Learning the model pararmeters using actual student data would improve the fidelity of the simulation.

In [66]:
simulation = simulate(AE_degree_plan, course_attempt_limit, students,
                      max_credits = max_credits,
                      performance_model = performance_model,
                      enrollment_model = enrollment_model,
                      duration = num_terms,
                      duration_lock = duration_lock,
                      stopouts = stopouts);
simulation_report(simulation, num_terms, course_passrate, max_credits, real_passrate)


------------ Simulation Report ------------
Aerospace Engineering, BS -- 2019-20 Degree Plan

-------- Simulation Statistics --------
Number of terms: 12
Max Credits per Term: 18
Max Course Attempts: 3
Number of Students: 1000
Preset Course Pass Rates: 90.0%

-------- Graduation Statistics --------
Number of Students Graduated: 587
Graduation Rate: 58.699999999999996%
Term Graduation Rates: 
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.012, 0.242, 0.49, 0.574, 0.587]
Average time to degree: 9.771431213955612 terms

-------- Stop out Statistics --------
Number of Students Stopped Out (Stopout Model Prediction + Reached Max Attempts): 412
Number of Students Reaching Max Attempts: 90
Stop-out Rate: 41.2%
Cumulative Term Stop-out Rates (including reached max course attempts students): 
[0.079, 0.199, 0.262, 0.345, 0.376, 0.393, 0.406, 0.406, 0.407, 0.408, 0.411, 0.412]

Cumulative Term Stop-out Rates (excluding reaching max course attempts students): 
[0.079, 0.199, 0.241, 0.289, 0.306, 0.315, 0

│ 24  │ 0.0%  │ 27.2% │ 57.3% │ 63.0% │ 63.6% │ 63.8% │ 63.8%  │ 63.8%  │
│ 25  │ 0.0%  │ 11.1% │ 38.8% │ 54.6% │ 58.9% │ 60.5% │ 60.9%  │ 61.0%  │
│ 26  │ 14.2% │ 28.1% │ 61.2% │ 65.8% │ 65.9% │ 65.9% │ 65.9%  │ 65.9%  │
│ 27  │ 9.2%  │ 14.9% │ 50.3% │ 64.1% │ 64.7% │ 64.8% │ 64.8%  │ 64.8%  │
│ 28  │ 98.2% │ 98.2% │ 98.5% │ 98.5% │ 98.5% │ 98.5% │ 98.5%  │ 98.5%  │
│ 29  │ 0.0%  │ 0.0%  │ 9.8%  │ 36.3% │ 51.9% │ 57.4% │ 59.2%  │ 59.6%  │
│ 30  │ 0.0%  │ 0.0%  │ 16.6% │ 47.7% │ 57.5% │ 60.1% │ 60.5%  │ 60.5%  │
│ 31  │ 0.0%  │ 0.0%  │ 8.9%  │ 46.8% │ 56.8% │ 59.7% │ 60.0%  │ 60.1%  │
│ 32  │ 0.0%  │ 0.0%  │ 4.7%  │ 40.9% │ 57.1% │ 59.5% │ 59.8%  │ 59.9%  │
│ 33  │ 99.8% │ 99.8% │ 99.8% │ 99.8% │ 99.8% │ 99.8% │ 99.8%  │ 99.8%  │
│ 34  │ 3.3%  │ 6.6%  │ 14.7% │ 38.3% │ 57.0% │ 61.3% │ 61.7%  │ 61.7%  │
│ 35  │ 0.0%  │ 0.0%  │ 0.0%  │ 8.3%  │ 45.1% │ 56.0% │ 58.9%  │ 59.3%  │
│ 36  │ 0.0%  │ 0.0%  │ 0.0%  │ 6.2%  │ 39.1% │ 54.1% │ 58.4%  │ 59.3%  │
│ 37  │ 0.0%  │ 0.0%  │ 0.0%  │ 6.5%  

Now, let's see what happens if we use the actual course pass rates in the computer science program.

In [67]:
set_passrates_from_csv(CS_degree_plan.curriculum.courses, "./Student_Grades_sp17_to_fall19.csv", course_passrate)
simulation = simulate(CS_degree_plan, course_attempt_limit, students,
                      max_credits = max_credits,
                      performance_model = performance_model,
                      enrollment_model = enrollment_model,
                      duration = num_terms,
                      duration_lock = duration_lock,
                      stopouts = stopouts);
simulation_report(simulation, num_terms, course_passrate, max_credits, real_passrate)


------------ Simulation Report ------------
Computer Science, BS -- 2019-20 Degree Plan

-------- Simulation Statistics --------
Number of terms: 12
Max Credits per Term: 18
Max Course Attempts: 3
Number of Students: 1000
Preset Course Pass Rates: 90.0%

-------- Graduation Statistics --------
Number of Students Graduated: 571
Graduation Rate: 57.099999999999994%
Term Graduation Rates: 
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.124, 0.444, 0.554, 0.57, 0.571]
Average time to degree: 9.03677758318739 terms

-------- Stop out Statistics --------
Number of Students Stopped Out (Stopout Model Prediction + Reached Max Attempts): 429
Number of Students Reaching Max Attempts: 149
Stop-out Rate: 42.9%
Cumulative Term Stop-out Rates (including reached max course attempts students): 
[0.076, 0.19, 0.279, 0.339, 0.367, 0.395, 0.413, 0.419, 0.422, 0.428, 0.429, 0.429]

Cumulative Term Stop-out Rates (excluding reaching max course attempts students): 
[0.076, 0.19, 0.21, 0.256, 0.269, 0.277, 0.28, 0.2

## 3. What-if Analyses

These simulation capabilities allow us to conduct what-if analyses around the impact that changes to curricular structure or instructional improvements will have on student success.  First, let's consider how changing the number of allowed attempts (from 2 to 3) would impact graduation rates in both of these programs.

In [68]:
course_attempt_limit = 3
simulation = simulate(AE_degree_plan, course_attempt_limit, students,
                      max_credits = max_credits,
                      performance_model = performance_model,
                      enrollment_model = enrollment_model,
                      duration = num_terms,
                      duration_lock = duration_lock,
                      stopouts = stopouts);
simulation_report(simulation, num_terms, course_passrate, max_credits, real_passrate)


------------ Simulation Report ------------
Aerospace Engineering, BS -- 2019-20 Degree Plan

-------- Simulation Statistics --------
Number of terms: 12
Max Credits per Term: 18
Max Course Attempts: 3
Number of Students: 1000
Preset Course Pass Rates: 90.0%

-------- Graduation Statistics --------
Number of Students Graduated: 571
Graduation Rate: 57.099999999999994%
Term Graduation Rates: 
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.014, 0.241, 0.475, 0.556, 0.571]
Average time to degree: 9.747810858143607 terms

-------- Stop out Statistics --------
Number of Students Stopped Out (Stopout Model Prediction + Reached Max Attempts): 427
Number of Students Reaching Max Attempts: 118
Stop-out Rate: 42.7%
Cumulative Term Stop-out Rates (including reached max course attempts students): 
[0.088, 0.201, 0.258, 0.332, 0.378, 0.398, 0.413, 0.418, 0.419, 0.421, 0.425, 0.427]

Cumulative Term Stop-out Rates (excluding reaching max course attempts students): 
[0.088, 0.201, 0.234, 0.276, 0.297, 0.303,

│ 22  │ 34.7%  │ 61.5%  │ 68.0%  │ 69.0%  │ 69.2%  │ 69.2%  │ 69.2%  │ 69.2%  │
│ 23  │ 0.0%   │ 6.2%   │ 38.3%  │ 55.1%  │ 60.4%  │ 61.3%  │ 61.3%  │ 61.3%  │
│ 24  │ 0.0%   │ 27.3%  │ 54.4%  │ 61.6%  │ 62.6%  │ 62.8%  │ 62.8%  │ 62.8%  │
│ 25  │ 0.0%   │ 10.5%  │ 36.1%  │ 52.4%  │ 58.6%  │ 60.3%  │ 60.7%  │ 60.7%  │
│ 26  │ 13.5%  │ 30.5%  │ 61.4%  │ 65.3%  │ 65.4%  │ 65.4%  │ 65.4%  │ 65.4%  │
│ 27  │ 9.2%   │ 15.0%  │ 51.0%  │ 63.2%  │ 63.8%  │ 63.8%  │ 63.8%  │ 63.8%  │
│ 28  │ 98.6%  │ 98.6%  │ 98.8%  │ 98.8%  │ 98.8%  │ 98.8%  │ 98.8%  │ 98.8%  │
│ 29  │ 0.0%   │ 0.0%   │ 9.1%   │ 32.5%  │ 50.5%  │ 56.4%  │ 58.6%  │ 59.3%  │
│ 30  │ 0.0%   │ 0.0%   │ 17.8%  │ 47.7%  │ 57.1%  │ 59.2%  │ 59.8%  │ 59.8%  │
│ 31  │ 0.0%   │ 0.0%   │ 9.8%   │ 44.9%  │ 57.5%  │ 58.9%  │ 59.5%  │ 59.6%  │
│ 32  │ 0.0%   │ 0.0%   │ 4.8%   │ 40.0%  │ 56.1%  │ 58.3%  │ 59.3%  │ 59.3%  │
│ 33  │ 100.0% │ 100.0% │ 100.0% │ 100.0% │ 100.0% │ 100.0% │ 100.0% │ 100.0% │
│ 34  │ 3.3%   │ 8.5%   │ 15.5%  │ 40.3%

In [69]:
simulation = simulate(CS_degree_plan, course_attempt_limit, students,
                      max_credits = max_credits,
                      performance_model = performance_model,
                      enrollment_model = enrollment_model,
                      duration = num_terms,
                      duration_lock = duration_lock,
                      stopouts = stopouts);
simulation_report(simulation, num_terms, course_passrate, max_credits, real_passrate)


------------ Simulation Report ------------
Computer Science, BS -- 2019-20 Degree Plan

-------- Simulation Statistics --------
Number of terms: 12
Max Credits per Term: 18
Max Course Attempts: 3
Number of Students: 1000
Preset Course Pass Rates: 90.0%

-------- Graduation Statistics --------
Number of Students Graduated: 541
Graduation Rate: 54.1%
Term Graduation Rates: 
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.125, 0.44, 0.532, 0.541, 0.0]
Average time to degree: 8.972273567467653 terms

-------- Stop out Statistics --------
Number of Students Stopped Out (Stopout Model Prediction + Reached Max Attempts): 459
Number of Students Reaching Max Attempts: 149
Stop-out Rate: 45.9%
Cumulative Term Stop-out Rates (including reached max course attempts students): 
[0.087, 0.21, 0.29, 0.352, 0.395, 0.423, 0.444, 0.453, 0.456, 0.459, 0.459, 0.0]

Cumulative Term Stop-out Rates (excluding reaching max course attempts students): 
[0.087, 0.21, 0.23, 0.275, 0.298, 0.307, 0.31, 0.31, 0.31, 0.31, 0.3

Next, notice that CSC 110 -- Intro to Computer Programming I is clearly a gateway course for the computer science program at the University of Arizona, and that is has a relatively low success rate. There are a few other courses with teh CSC prefix that also have low success rates. What would happen if the instruction and instructional support were changed in a way that enabled the students taking CSC 110, CSC 120 and CSC 353 to obtain a 90% pass rate? 

In [70]:
convert_ids(CS_degree_plan.curriculum)
csc110 = course(CS_degree_plan.curriculum, "CSC", "110", "Intro to Computer Programming I", "")
csc120 = course(CS_degree_plan.curriculum, "CSC", "120", "Intro to Computer Programming II", "")
csc353 = course(CS_degree_plan.curriculum, "CSC", "352", "Systems Programming & Unix", "")
csc110.passrate = csc120.passrate = csc353.passrate = 0.9;

In [71]:
simulation = simulate(CS_degree_plan, course_attempt_limit, students,
                      max_credits = max_credits,
                      performance_model = performance_model,
                      enrollment_model = enrollment_model,
                      duration = num_terms,
                      duration_lock = duration_lock,
                      stopouts = stopouts);
simulation_report(simulation, num_terms, course_passrate, max_credits, real_passrate)


------------ Simulation Report ------------
Computer Science, BS -- 2019-20 Degree Plan

-------- Simulation Statistics --------
Number of terms: 12
Max Credits per Term: 18
Max Course Attempts: 3
Number of Students: 1000
Preset Course Pass Rates: 90.0%

-------- Graduation Statistics --------
Number of Students Graduated: 633
Graduation Rate: 63.3%
Term Graduation Rates: 
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.174, 0.529, 0.622, 0.633, 0.0]
Average time to degree: 8.906793048973144 terms

-------- Stop out Statistics --------
Number of Students Stopped Out (Stopout Model Prediction + Reached Max Attempts): 367
Number of Students Reaching Max Attempts: 68
Stop-out Rate: 36.7%
Cumulative Term Stop-out Rates (including reached max course attempts students): 
[0.065, 0.179, 0.221, 0.274, 0.325, 0.339, 0.357, 0.358, 0.362, 0.366, 0.367, 0.0]

Cumulative Term Stop-out Rates (excluding reaching max course attempts students): 
[0.065, 0.179, 0.209, 0.257, 0.286, 0.294, 0.299, 0.299, 0.299, 0.

# References

Heileman, G. L., Abdallah, C.T., Slim, A., and Hickman, M. (2018). Curricular analytics: A framework for quantifying the impact of curricular reforms and pedagogical innovations. www.arXiv.org, arXiv:1811.09676 [cs.CY].

Hickman, M. (2014). Development of a Curriculum Analysis and Simulation Library with Applications in Curricular Analytics. MS thesis, University of New Mexico,
Albuquerque, NM.